In [ ]:
from matplotlib import pyplot as plt
from collections import Counter
import seaborn as sns
import pandas as pd
import psycopg2
import pickle
import requests
import os
import re

import warnings

warnings.filterwarnings("ignore")

%load_ext nb_black

In [ ]:
# https://github.com/openeventdata/mordecai
#  $ python -m spacy download en_core_web_lg
#  $ docker pull elasticsearch:5.5.2
#  $ wget https://andrewhalterman.com/files/geonames_index.tar.gz
#  $ docker run -d -p 127.0.0.1:9200:9200 -v $(pwd)/geonames_index/:/usr/share/elasticsearch/data elasticsearch:5.5.2
from mordecai import Geoparser

# https://github.com/somnathrakshit/geograpy3
#  > This wasn't very good.
# import geograpy

In [ ]:
def connect():
    conn = psycopg2.connect(
        user="postgres",
        password=os.environ.get("POSTGRES_PASS", ""),
        host="localhost",
        port=5432,
        database="venmo",
    )
    return conn

In [ ]:
geo = Geoparser()

In [ ]:
mordecai_tokens = Counter()
meta = Counter()

conn = connect()
with conn.cursor(name="exploration") as cursor:
    cursor.itersize = 2000
    cursor.execute("SELECT * FROM transactions")
    for i, row in enumerate(cursor):
        if i % 10000 == 0:
            # checkpoint
            print("Row", i)
            with open("mordecai_tokens.pkl", "wb") as f:
                pickle.dump(mordecai_tokens, f)
            with open("meta.pkl", "wb") as f:
                pickle.dump(meta, f)
        msg = row[1]
        msg = re.sub(r"[^\w\d_\- ]", "", msg).strip()
        meta["msgs"] += 1
        if len(msg) == 0:
            continue
        meta["msgs_processed"] += 1

        g = geo.geoparse(msg)
        if len(g) != 0:
            for w in [item['word'] for item in g]:
                mordecai_tokens[w] += 1

conn.close()

In [ ]:
# Checkpoint
with open("mordecai_tokens.pkl", "rb") as f:
    mordecai_tokens_saved = pickle.load(f)
with open("meta.pkl", "rb") as f:
    meta_saved = pickle.load(f)
meta_saved

In [ ]:
# 71%
meta_saved["msgs_processed"] / meta_saved["msgs"]

In [ ]:
# 1%
sum(mordecai_tokens_saved.values()) / meta_saved["msgs"]

In [ ]:
REPLACE_MORDECAI_TOKENS = {
    "Cali": "California", "jersey": "Jersey", "baja": "Baja", "Vegas": "Las Vegas"
}
IGNORE_MORDECAI_TOKENS = set([
    "Wendys", "Bc", "Hookers", "foooood", "Santa", "Easter", "Playa", "Yeehaw", "Zelle", "AZ", "Tn", "Anotha",
    "Rv", "RV", "FL", "NY", "Sydney", "Turkey", "Brittany", "Cleaning", "Pussy", "Daves", "Charlotte", "NC", "Beach"
])
N = 50

# Handle aliases
for repl, token in REPLACE_MORDECAI_TOKENS.items():
    if repl in mordecai_tokens_saved:
        cnt = mordecai_tokens_saved[repl]
        del mordecai_tokens_saved[repl]
        mordecai_tokens_saved[token] += cnt

tokens = []
counts = []
for token, cnt in mordecai_tokens_saved.most_common(N * 2):
    if token in IGNORE_MORDECAI_TOKENS:
        continue
    tokens.append(token)
    counts.append(cnt)
    if len(tokens) == N:
        break
plt.figure(figsize=(15, N // 5))
sns.barplot(y=tokens, x=counts, orient="h").set_title(
    "Mordecai Keywords (of {} msgs)".format(meta["msgs"])
)
# Tokens for other countries often used as adjective, "China virus" "China town" "China food"